我们现在开始分析下试题数据的主题分布，首先导入一些库

In [77]:
from gensim import corpora, models
import pandas as pd

停用词还是使用上次的文件

In [78]:
stop_words = pd.read_csv('../exec_1/stopwords/stopwords2.txt', encoding='utf-8',sep='\t', header=None,quoting=3)

id_data是我整理好的数据

In [79]:
df = pd.read_csv('./data/id_data.txt', encoding='utf-8',sep='\t',quoting=3)

In [80]:
df.head

<bound method NDFrame.head of              id                                               segs
0          1_id  2015 年 6 月 29 日 郑州市 召开 社会 信用 体系 建设 工作 会议 总结 近年...
1         10_id  2012 年 11 月 长江日报 报道 目前 武汉 多家 旅行社 规定 云南 特价 团 的 ...
2        100_id  2012 年 4 月 中国 青年网 中国 青少年 网 协会 联合 向 广大 青少年 发出 青...
3       1000_id  回眸 十一五 改革开放 纵深 发展 经济社会 发展 又 好 又 快 十二五 是 全面 建设 ...
4      10000_id  动物细胞 与 高等植物 细胞 的 有丝分裂 不同之处 为 纺锤体 形成 方式 不同 染色体 ...
...         ...                                                ...
17912   9995_id  现 提供 新 配制 的 斐林 试剂 甲液 1 g mLNaOH 溶液 乙液 05 g mLC...
17913   9996_id  下列 关于 糖类 和 脂质 的 叙述 正确 的 说法 有 分别 含有 5 个 和 12 个 ...
17914   9997_id  植物 细胞 表现 出 全能性 的 必要条件 是 A . 给予 适宜 的 营养 和 外界 条件...
17915   9998_id  有关 细胞 全能性 的 叙述 中 错误 的 是 A . 受精卵 在 自然 条件 下 能够 使...
17916   9999_id  下列 有关 细胞 全能性 的 叙述 正确 的 是 A . 组织培养 繁育 花卉 所 利用 的...

[17917 rows x 2 columns]>

In [81]:
df=df.dropna()

In [82]:
docs = df['segs']

In [83]:
docs.head()

0    2015 年 6 月 29 日 郑州市 召开 社会 信用 体系 建设 工作 会议 总结 近年...
1    2012 年 11 月 长江日报 报道 目前 武汉 多家 旅行社 规定 云南 特价 团 的 ...
2    2012 年 4 月 中国 青年网 中国 青少年 网 协会 联合 向 广大 青少年 发出 青...
3    回眸 十一五 改革开放 纵深 发展 经济社会 发展 又 好 又 快 十二五 是 全面 建设 ...
4    动物细胞 与 高等植物 细胞 的 有丝分裂 不同之处 为 纺锤体 形成 方式 不同 染色体 ...
Name: segs, dtype: object

In [87]:
doclist=docs.values
print(doclist[0])
print(doclist[1])
print(doclist[2])

2015 年 6 月 29 日 郑州市 召开 社会 信用 体系 建设 工作 会议 总结 近年来 的 社会 信用 体系 建设 情况 安排 部署 下一阶段 工作 建设 社会 信用 体系 需要 消费者 加大 对 失信 行为 的 打击 力度 企业经营者 遵循 诚信 原则 消费者 敢于 拿 起 法律 武器 维护 自身 合法权益 充分发挥 市场 对 资源配置 的 决定 作用
2012 年 11 月 长江日报 报道 目前 武汉 多家 旅行社 规定 云南 特价 团 的 年龄 必须 在 28 岁 至 54 岁 之间 年龄 达 不到 要求 需 另外 支付 300 元 究其原因 竟是 85 后 熟悉 网络 报团 之前 多 会 在 网络 了解 市场行情 与 价位 跟 团 到 了 购物 景点 多 不会 出手 购物 导游 拿 不到 提成 当下 团队 旅游 市场 的 现状 是 由于 竞争 激烈 旅行社 多 采取 低 报价 手段 吸引 客源 然后 从 旅游 质量 打折 频繁 安排 购物 等 环节 中 补足 为此 某市 旅游 行业协会 通过 评选 推荐 诚信 旅行社 树立 诚信 旅行社 企业形象 带动 整个 行业 共同 走上 诚信 经营 的 道路 创建 和谐 健康 的 旅游 市场 1. 如何 认识 某些 旅行社 的 做法 1. 材料 中 某市 旅游 行业协会 的 做法 给 我们 什么 启示
2012 年 4 月 中国 青年网 中国 青少年 网 协会 联合 向 广大 青少年 发出 青少年 坚决 不信谣不 传谣 倡议书 希望 广大 青少年 自觉 做 网络 健康 环境 的 维护者 说明 网络 技术 的 发展 带来 了 哪些 伦理 问题 网络 自由 的 滥用 问题 网络安全 问题 网络 谣言 严重 侵犯 公民 权益 损害 公共利益 危害 国家 安全 和 社会 稳定 必须 坚决 抵制 抵制 网络 谣言 推动 网络 行业 健康 可 持续 发展 是 青少年 应有 的 公民 意识 和 政治 素养 青少年 要 树立 法律意识 严格遵守 互联网 法律法规 自觉 远离 网络 谣言 . A


将数据处理成我们要的样子

In [88]:
texts = [[word for word in doc.split(' ') if word not in stop_words and len(word) > 1] for doc in doclist]

In [89]:
texts[0]

['2015',
 '29',
 '郑州市',
 '召开',
 '社会',
 '信用',
 '体系',
 '建设',
 '工作',
 '会议',
 '总结',
 '近年来',
 '社会',
 '信用',
 '体系',
 '建设',
 '情况',
 '安排',
 '部署',
 '下一阶段',
 '工作',
 '建设',
 '社会',
 '信用',
 '体系',
 '需要',
 '消费者',
 '加大',
 '失信',
 '行为',
 '打击',
 '力度',
 '企业经营者',
 '遵循',
 '诚信',
 '原则',
 '消费者',
 '敢于',
 '法律',
 '武器',
 '维护',
 '自身',
 '合法权益',
 '充分发挥',
 '市场',
 '资源配置',
 '决定',
 '作用']

词袋模型处理

In [90]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [92]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 3),
 (6, 1),
 (7, 3),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 2),
 (18, 1),
 (19, 3),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 3),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1)]

开始训练LDA主题模型

In [97]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5)

In [98]:
lda.print_topic(1, topn=20)

'0.019*"太阳" + 0.017*"下列" + 0.014*"正确" + 0.012*"叙述" + 0.011*"公转" + 0.010*"昼夜" + 0.008*"直射" + 0.008*"关于" + 0.007*"正午" + 0.007*"太阳系" + 0.007*"地壳" + 0.007*"物质" + 0.007*"移动" + 0.007*"运动" + 0.007*"天体" + 0.006*"北半球" + 0.006*"变化" + 0.006*"生命" + 0.006*"火星" + 0.006*"太阳辐射"'

In [99]:
lda.print_topics(num_topics=20, num_words=20)

[(0,
  '0.036*"细胞" + 0.018*"染色体" + 0.014*"下列" + 0.014*"相同" + 0.014*"基因" + 0.011*"正确" + 0.009*"叙述" + 0.008*"蛋白质" + 0.008*"DNA" + 0.007*"生物" + 0.007*"基因型" + 0.006*"有关" + 0.006*"一个" + 0.006*"分子" + 0.006*"不同" + 0.006*"关于" + 0.005*"位于" + 0.005*"发生" + 0.005*"细胞核" + 0.005*"过程"'),
 (1,
  '0.019*"太阳" + 0.017*"下列" + 0.014*"正确" + 0.012*"叙述" + 0.011*"公转" + 0.010*"昼夜" + 0.008*"直射" + 0.008*"关于" + 0.007*"正午" + 0.007*"太阳系" + 0.007*"地壳" + 0.007*"物质" + 0.007*"移动" + 0.007*"运动" + 0.007*"天体" + 0.006*"北半球" + 0.006*"变化" + 0.006*"生命" + 0.006*"火星" + 0.006*"太阳辐射"'),
 (2,
  '0.020*"地球" + 0.011*"经济" + 0.011*"发展" + 0.010*"中国" + 0.008*"材料" + 0.007*"人口" + 0.007*"我国" + 0.007*"城市" + 0.006*"国家" + 0.006*"地区" + 0.006*"北京" + 0.005*"主要" + 0.004*"地方" + 0.004*"问题" + 0.004*"下列" + 0.004*"时间" + 0.004*"现象" + 0.004*"社会" + 0.004*"生活" + 0.003*"影响"'),
 (3,
  '0.012*"太阳" + 0.009*"社会" + 0.007*"发展" + 0.006*"经济" + 0.006*"下列" + 0.006*"细胞" + 0.006*"市场" + 0.005*"社会主义" + 0.005*"材料" + 0.005*"英国" + 0.005*"政府" + 0.004*"分配" + 0.004*"历史" + 0.004

In [104]:
bow_sample =[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 3),
 (6, 1),
 (7, 3),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 2),
 (18, 1),
 (19, 3),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 3),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1)]

In [103]:
lda.get_document_topics(bow_sample)

[(0, 0.0678866),
 (1, 0.72788554),
 (2, 0.06763608),
 (3, 0.06833906),
 (4, 0.068252765)]